<a href="https://colab.research.google.com/github/mmelicher91/ITSD_rozpis/blob/main/itsd_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import numpy as np
import pandas as pd
import calendar
from tabulate import tabulate

###################### Svátky 2023: [měsíc, den] ###############################
input("Stiskni ENTER |")
print("|-------------|")
with open("data/osoby.csv", "r", encoding="cp1250") as soubor: 
  soubor=soubor.readlines()
  osoby=[]
  for kazdy_radek in soubor:
    rozstipej = kazdy_radek.strip().split(';')
    osoby.append(rozstipej)

xp=[]
yp=[]
barveni={} # generování dict pro barveni v pandas
for i in range(len(osoby)):
  xp.append(osoby[i][0])
  mezi_y=osoby[i][1]
  uloz1=dict.fromkeys(xp,mezi_y)
  barveni.update(uloz1)
  xp=[]
  yp=[]

with open("data/svatky.csv", "r", encoding="cp1250") as soubor2: 
  soubor2=soubor2.readlines()
  svatky=[]
  for kazdy_radek in soubor2:
    rozstipej = kazdy_radek.strip().split(';')
    svatky.append(rozstipej)

######################### Nastavení osob #######################################
    

########################### Statická data ######################################
dny_v_tydnu= ("Po ","Út ","St ","Čt ","Pá ","So ","Ne ")
nazvy_mesicu =("Leden","Únor","Březen", "Duben", "Květen", "Červen", "Červenec", "Srpen", "Září", "Říjen", "Listopad", "Prosinec")
nadpis = ("DEN","RANO","ODPO"," ","POH","   ")


################################################################################
################################# Clovek #######################################
################################################################################

class Clovek():
  # Generuje předchůdce a následovníka, rozlišuje lichý vs sudý počet osob
  def __init__(self,p):
    self.index=p
    self.znacka=osoby[p][0]
    self.barva=osoby[p][1]
    # Podmínka: LICHÝ vs SUDÝ počet osob
    if len(osoby)%2==0:
      if p%2==0: 
        # Sudý řádky posunu o +1 hodnotu vpřed
        self.predchudce=osoby[p-len(osoby)+1][0]
        # Podmínka: NASLEDUJICI: poslední sudý řádek se odkáže na začátek osoby[index=0]
        if len(osoby)-2==p:  
          self.nasledujici=osoby[len(osoby)-p-2][0]
        else: 
          # Není poslední sudý řádek .. skáču o +3 indexy
          self.nasledujici=osoby[p+3-len(osoby)][0]      
      else: 
        # Lichý řádky: Posunu se o -1 hodnotu zpět
        self.predchudce=osoby[p-len(osoby)-1][0]
        # Podmínka: NASLEDUJICI: poslední lichý řádek se odkáže na první lichý záznam (tj. začátek osoby[index=1])
        if len(osoby)-1==p: 
          self.nasledujici=osoby[len(osoby)-p][0]
        else:
          self.nasledujici=osoby[p+1-len(osoby)][0]
    else: 
      # Lichý počet osob: řazení je automatické, netřeba ošetřovat
      self.predchudce=osoby[p-1][0]
      self.nasledujici=osoby[p-len(osoby)+1][0]
    self.poradi_poho=osoby[p][2]
################################################################################
################################# Rozpis #######################################
################################################################################

class Rozpis:
  def __init__(self,p):
    self.index=p
    self.rok=rok
    self.mesic=p+1
    self.nazev=nazvy_mesicu[p]
    kal = calendar.monthrange(self.rok,self.mesic)
    self.prvni_den = kal[0]
    self.jmeno_dne = dny_v_tydnu[self.prvni_den]
    self.delka_dnu = kal[1]
    
    pocet=[]
    for _ in range(self.delka_dnu):
      pocet.append(_+1)
    self.pocet=pocet

    self.matice= np.zeros((self.delka_dnu,6),dtype=object)
    
    for k in range(len(svatky)):
      if int(svatky[k][0]) == self.mesic:
        for jk in range(self.delka_dnu):
          if int(svatky[k][1]) == self.pocet[jk]:
            self.matice[jk,1]="X"
            self.matice[jk,2]="X"
 
################################################################################ 
sluzba=[]
def rozpis_dnu(den,poc,posledni,poho):
  poradi=1 # rozhoduje o použití PREDCHUDCE (poradi=-1) x NASLEDOVNIK (poradi=1)
  for k in range(len(osoby)):
    if poho == osoby[k][0]:
    #Pokud poslední POHO v měsíci je neděle, tak posune o index dál
      if (k == len(osoby)-1) and (den==0):
        idx=0
        break
      elif (k == len(osoby)-1) and (den!=0):
        idx=k
      elif (k < len(osoby)-1) and (den!=0):
        idx=k
      else:
        idx=k+1

  for i in range(poc):
    for j in range(2):
      mesic[x].matice[i,0]=dny_v_tydnu[den] #<-- vypsání dnů
      if (5 <= den <= 6):
        mesic[x].matice[i,j+1]="" # So/Ne Ráno/Odpo
        mesic[x].matice[i,5]="" # prázdný sloupec
      else:
        posledni=predej(posledni,poradi)
        posl_odpo=posledni
        if mesic[x].matice[i,j+1]=="X":
          mesic[x].matice[i,j+1]=""
          mesic[x].matice[i,5]="sv"          
        else:
          mesic[x].matice[i,j+1]=posledni
          mesic[x].matice[i,5]=""
        # poradi se meni pouze u sudych osob
        if len(osoby)%2==0:
          poradi*=-1
    
    mesic[x].matice[i,4]=osoby[idx][0]
    mesic[x].matice[i,3]="" # prázdný sloupec
    posl_poho=osoby[idx][0]

    if den == 6:
      den=0
      if idx == len(osoby)-1:
        idx=0
      else:
        idx+=1
    else:
      den+=1
  xp=["Poslední odpolední","Poslední pohotovost"]
  sluzba.append(posl_odpo)
  sluzba.append(posl_poho)
  sluzba2={}
  sluzba2=dict.fromkeys(xp,sluzba)
  return sluzba2

def predej(posledni,poradi):
  for kl in range(len(osoby)):
    if posledni==clovek[kl].znacka:
      if poradi==-1:
        posledni=clovek[kl].predchudce
      else:
        posledni=clovek[kl].nasledujici
      break  
  return posledni

############################# Barveni PANDAS ###################################
def obarvi(col, pattern_map, default=''):
  return np.select(
      [col.str.contains(k, na=False) for k in pattern_map.keys()],
      [f'background-color: {v}' for v in pattern_map.values()],
      default=default, 
    ).astype(str)
################################################################################
obj = calendar.Calendar()
######################### Vyprázdněný při spuštění #############################
mesic=[]
clovek=[]
pocet=[]
y=[]
sluzba=[]
################################################################################
rok=int(input("Zadej rok: "))
start=True
x=int(input("Zadej měsíc: "))-1
print(f"+++ Nabídka zkratek: {([osoby[_][0] for _ in range(len(osoby))])} +++")
posledni=input("Zadej poslední odpolední službu: ").upper()
poho=input("Zadej poslední pohotovost: ").upper()

for p in range(len(osoby)):
  clvk=Clovek(p)
  clovek.append(clvk)
for p in range(12):
    mesic.append(Rozpis(p))

poc=mesic[x].delka_dnu
den=mesic[x].prvni_den
data = pd.DataFrame(mesic[x].matice, index=mesic[x].pocet, columns=nadpis)
rozpis_dnu(den, poc, posledni,poho)

while start == True:
  kontrola= True
  while kontrola == True:
    nabidka=[["Akce","Popis"],[0, "Textový výstup"],[1,"Export do excelu"],[2,"Export do html"],[7, "Informace o lidech"],[8, "Změnit zadání"],[9, "Ukončit program"]]
    print(tabulate(nabidka, headers="firstrow", tablefmt="rst", showindex=False, numalign="center"))
    otazka=int(input(">> "))
    print("")
    if otazka == 0:
      print(tabulate(mesic[x].matice, tablefmt="github", showindex=mesic[x].pocet))
      print("")
    elif otazka == 1:
      data.style.apply(obarvi, pattern_map=barveni, subset=["RANO","ODPO","POH"]).to_excel("report.xlsx")
      print("Exportuji do excelu...")
    elif otazka == 2:
      data.style.apply(obarvi, pattern_map=barveni, subset=["RANO","ODPO","POH"]).to_html("report.html")
      print("Exportuji do html...")
    elif otazka == 6:
      #print(f"{mesic[x].nazev} {mesic[x].rok}")
      display(data.style.apply(obarvi,pattern_map=barveni,subset=["RANO","ODPO","POH"]))
      print("\n")
    elif otazka == 7:
      for i in range(len(osoby)):
        print(vars(clovek[i]))
      print("\n??? Vysvětlení:")
      print(" ? Predchudce: předcházející ranní šlužba")
      print(" ? Nasledujici: následující ranní služba")
    elif otazka == 8:
      kontrola==False
      xp=["Poslední odpolední","Poslední pohotovost"]
      datumovka=[mesic[x].nazev,mesic[x].rok]
      sluzba=[["Poslední odpolední",sluzba[0]],["Poslední pohotovost",sluzba[1]]]
      print(tabulate(sluzba, headers=datumovka, tablefmt="psql", showindex=False))
      x=int(input("Zadej měsíc: "))-1
      print(f"+++ Nabídka zkratek: {([osoby[_][0] for _ in range(len(osoby))])} +++")
      posledni=input("Zadej poslední odpolední službu: ").upper()
      poho=input("Zadej poslední pohotovost: ").upper()
      mesic=[]
      sluzba=[]
      for p in range(12):
        mesic.append(Rozpis(p))
      poc=mesic[x].delka_dnu
      den=mesic[x].prvni_den
      data = pd.DataFrame(mesic[x].matice, index=mesic[x].pocet, columns=nadpis)
      rozpis_dnu(den, poc, posledni,poho)


    else:
      print("\nUkončuji program...")
      start=False
      break

Stiskni ENTER |
|-------------|
Zadej rok: 2023
Zadej měsíc: 5
+++ Nabídka zkratek: ['ŠTO', 'LAN', 'PAL', 'ITIN'] +++
Zadej poslední odpolední službu: lan
Zadej poslední pohotovost: što
======  ==================
 Akce   Popis
======  ==================
  0     Textový výstup
  1     Export do excelu
  2     Export do html
  7     Informace o lidech
  8     Změnit zadání
  9     Ukončit program
======  ==================
>> 0

|----|----|------|------|--|------|----|
|  1 | Po |      |      |  | LAN  | sv |
|  2 | Út | LAN  | ŠTO  |  | LAN  |    |
|  3 | St | ITIN | PAL  |  | LAN  |    |
|  4 | Čt | ŠTO  | LAN  |  | LAN  |    |
|  5 | Pá | PAL  | ITIN |  | LAN  |    |
|  6 | So |      |      |  | LAN  |    |
|  7 | Ne |      |      |  | LAN  |    |
|  8 | Po |      |      |  | PAL  | sv |
|  9 | Út | ITIN | PAL  |  | PAL  |    |
| 10 | St | ŠTO  | LAN  |  | PAL  |    |
| 11 | Čt | PAL  | ITIN |  | PAL  |    |
| 12 | Pá | LAN  | ŠTO  |  | PAL  |    |
| 13 | So |      |      |  | PAL  | 

,DEN,RANO,ODPO,,POH,
1,Čt,ŠTO,LAN,,LAN,
2,Pá,PAL,ITIN,,LAN,
3,So,,,,LAN,
4,Ne,,,,LAN,
5,Po,LAN,ŠTO,,PAL,
6,Út,ITIN,PAL,,PAL,
7,St,ŠTO,LAN,,PAL,
8,Čt,PAL,ITIN,,PAL,
9,Pá,LAN,ŠTO,,PAL,
10,So,,,,PAL,




======  ==================
 Akce   Popis
======  ==================
  0     Textový výstup
  1     Export do excelu
  2     Export do html
  7     Informace o lidech
  8     Změnit zadání
  9     Ukončit program
======  ==================
>> 9


Ukončuji program...
